In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('sensor_log.csv')

# ---------------------------------------------------------
# Part 1: Show rows where temperature_c is missing
# ---------------------------------------------------------
# We use .isnull() to find the NaNs and then use that to filter the dataframe
missing_temp = df[df['temperature_c'].isnull()]

print("Exercise One:")
print("1. Rows with missing Temperature:")
print(missing_temp)
print("-" * 30)

# ---------------------------------------------------------
# Part 2: Show rows where humidity_pct is missing
# ---------------------------------------------------------
missing_humidity = df[df['humidity_pct'].isnull()]

print("2. Rows with missing Humidity:")
print(missing_humidity)
print("-" * 30)

# ---------------------------------------------------------
# Part 3: Which column has the highest % of missing values?
# ---------------------------------------------------------
# Calculate the percentage: (Count of Nulls / Total Rows) * 100
missing_percent = (df.isnull().sum() / len(df)) * 100

print("3. Percentage of missing values per column:")
print(missing_percent)

# Find the maximum automatically
highest_col = missing_percent.idxmax()
highest_val = missing_percent.max()

print(f"\nANSWER: The column '{highest_col}' has the highest missing data ({highest_val:.1f}%).")

Exercise One:
1. Rows with missing Temperature:
             timestamp  temperature_c  humidity_pct  voltage_v
3  2025-10-01 08:00:30            NaN          54.9       3.68
8  2025-10-01 08:08:00            NaN          55.0       3.64
------------------------------
2. Rows with missing Humidity:
             timestamp  temperature_c  humidity_pct  voltage_v
5  2025-10-01 08:02:15           25.1           NaN       3.67
------------------------------
3. Percentage of missing values per column:
timestamp         0.0
temperature_c    20.0
humidity_pct     10.0
voltage_v        10.0
dtype: float64

ANSWER: The column 'temperature_c' has the highest missing data (20.0%).


In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv('sensor_log.csv')

# Select only numeric columns to avoid errors with the timestamp
numeric_cols = ['temperature_c', 'humidity_pct', 'voltage_v']

# ---------------------------------------------------------
# Part 1: Mean Imputation (Recap for comparison)
# ---------------------------------------------------------
df_mean = df.copy()
df_mean[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# ---------------------------------------------------------
# Part 2: Median Imputation (The Task)
# ---------------------------------------------------------
# Create a copy called df_median
df_median = df.copy()

# Fill missing values with the median of each column
df_median[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# ---------------------------------------------------------
# Part 3: Compare Results
# ---------------------------------------------------------
print("Exercise Two:")
print("Values used for filling:")
print(f"Mean Temp:   {df['temperature_c'].mean():.4f}")
print(f"Median Temp: {df['temperature_c'].median():.4f}")

print("\n--- Example: Row 3 (Missing Temperature) ---")
print(f"Mean Imputation result:   {df_mean.loc[3, 'temperature_c']:.4f}")
print(f"Median Imputation result: {df_median.loc[3, 'temperature_c']:.4f}")

Exercise Two:
Values used for filling:
Mean Temp:   25.0750
Median Temp: 25.0000

--- Example: Row 3 (Missing Temperature) ---
Mean Imputation result:   25.0750
Median Imputation result: 25.0000


The Median-based imputation (df_median) is more robust to extreme values (outliers).

Why? The Mean is calculated by adding all values, so one huge outlier (e.g., a sensor error reading 100°C) would pull the average up drastically. The Median is simply the "middle" value, so it ignores how big or small the extreme values are at the edges.

In this dataset: The difference is small (Mean 25.075 vs Median 25.000) because our data is fairly consistent. However, if we had a spike, the Median would be the safer choice.

In [5]:
import pandas as pd

# Load the dataset
df = pd.read_csv('sensor_log.csv')

# ---------------------------------------------------------
# Step 1: Create df_ts (Time Series DataFrame)
# ---------------------------------------------------------
# Convert timestamp column to actual datetime objects
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Set it as the index
df_ts = df.set_index('timestamp')

# ---------------------------------------------------------
# Step 2: Apply the 3 Methods
# ---------------------------------------------------------
# 1. Forward Fill
df_ffill = df_ts.ffill()

# 2. Backward Fill
df_bfill = df_ts.bfill()

# 3. Interpolation (Time-weighted)
# 'method="time"' is smarter: it looks at the actual time gap.
# If the gap is 10 seconds vs 1 hour, it adjusts the line accordingly.
df_interp = df_ts.interpolate(method='time')

# ---------------------------------------------------------
# Step 3: Side-by-Side Comparison
# ---------------------------------------------------------
# Let's focus on Temperature where we know data is missing
# Create a new dataframe just to show the comparison columns
comparison_df = pd.DataFrame({
    'Original': df_ts['temperature_c'],
    'Forward Fill': df_ffill['temperature_c'],
    'Backward Fill': df_bfill['temperature_c'],
    'Interpolation': df_interp['temperature_c']
})

# Show only the rows that were originally missing
print("Exercise Three")
print("--- Comparison of Imputation Methods (Temperature) ---")
print(comparison_df[comparison_df['Original'].isnull()])

Exercise Three
--- Comparison of Imputation Methods (Temperature) ---
                     Original  Forward Fill  Backward Fill  Interpolation
timestamp                                                                
2025-10-01 08:00:30       NaN          24.6           24.9      24.675000
2025-10-01 08:08:00       NaN          25.5           26.0      25.777778


Discussion: Which method is most reasonable?

The Winner: Linear Interpolation (method='time')

Why? Sensor data (Temperature, Humidity) changes gradually in the physical world. It rarely "jumps" instantly (which Forward Fill implies) or teleports from the future (Backward Fill).

Evidence: Look at the row for 08:00:30:

Forward Fill: Kept it at 24.6 (assuming no change).

Backward Fill: Jumped to 24.9 (assuming it was already hot).

Interpolation: Calculated 24.675, which is a smooth transition between the previous reading (24.6) and the next one (24.9). This is physically the most likely value.

In [7]:
import pandas as pd

# ==========================================
# STEP 1: LOAD THE DATA
# ==========================================
df = pd.read_csv('sensor_log.csv')

# ==========================================
# STEP 2: SUMMARISE MISSING VALUES
# ==========================================
print("--- Missing Values Summary ---")
# Create a dataframe to show counts and percentages nicely
missing_summary = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Percentage (%)': (df.isnull().sum() / len(df)) * 100
})
print(missing_summary)

# ==========================================
# STEP 3 & 4: DECISION & IMPUTATION
# ==========================================
# Strategy: Linear Interpolation (Time-weighted)
# Why? Because dropping 20% of our small dataset is bad, and sensor data is continuous.

# Convert to datetime for accurate time-based interpolation
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_clean = df.set_index('timestamp').interpolate(method='time').reset_index()

print("\n--- Imputation Successful ---")
print("Missing values after cleanup:")
print(df_clean.isnull().sum())

# ==========================================
# STEP 5: COMPARE STATISTICS
# ==========================================
# We compare the 'temperature_c' column before and after
stats_before = df['temperature_c'].describe()[['mean', 'min', 'max']]
stats_after = df_clean['temperature_c'].describe()[['mean', 'min', 'max']]

comparison_df = pd.DataFrame({
    'Before Imputation': stats_before,
    'After Imputation': stats_after
})

print("\n--- Statistics Comparison (Temperature) ---")
print(comparison_df)

--- Missing Values Summary ---
               Missing Count  Percentage (%)
timestamp                  0             0.0
temperature_c              2            20.0
humidity_pct               1            10.0
voltage_v                  1            10.0

--- Imputation Successful ---
Missing values after cleanup:
timestamp        0
temperature_c    0
humidity_pct     0
voltage_v        0
dtype: int64

--- Statistics Comparison (Temperature) ---
      Before Imputation  After Imputation
mean             25.075         25.105278
min              24.500         24.500000
max              26.000         26.000000
